<a href="https://colab.research.google.com/github/Daisuke0209/DataAccess/blob/master/scrapying_boatdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 競艇データの自動スクレイピング

## Google Driveとのマウントを取る

In [1]:
#Google Driveのマウントを取る
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## ライブラリのインポート

In [0]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import csv
import os
import itertools
import datetime as dt

## Google Driveの所定フォルダに出力用ファイル作成

In [0]:
os.chdir("/content/gdrive/My Drive/Datasets/boat_data")
csvFile = open("boat_file.csv",'wt',newline ='',encoding="utf-8")
writer=csv.writer(csvFile)

## 出力ファイルにインデックス記入

In [4]:
index = []
index.extend(['Date','Place','Race',
              '6th-rank','6th-num',
              '5th-rank','5th-num',
              '4th-rank','4th-num',
              '3rd-rank','3rd-num',
              '2nd-rank','2nd-num',
              '1st-rank','1st-num',
              '6th-ratio','5th-ratio','4th-ratio','3rd-ratio','2nd-ratio','1st-ratio',
              '6th-ratio2','5th-ratio2','4th-ratio2','3rd-ratio2','2nd-ratio2','1st-ratio2',
              '6th-ratioj','5th-ratioj','4th-ratioj','3rd-ratioj','2nd-ratioj','1st-ratioj',
              '6th-ratioj2','5th-ratioj2','4th-ratioj2','3rd-ratioj2','2nd-ratioj2','1st-ratioj2',
              '6th-st','5th-st','4th-st','3rd-st','2nd-st','1st-st',
              '6th-mratio','5th-mratio','4th-mratio','3rd-mratio','2nd-mratio','1st-mratio',
              '6th-bratio','5th-bratio','4th-bratio','3rd-bratio','2nd-bratio','1st-bratio',
              '1st-exb','2nd-exb','3rd-exb','4th-exb','5th-exb','6th-exb', 
              '1st','2nd','3rd','4th','5th','6th',
              '1-odds','2-odds','3-odds','4-odds','5-odds','6-odds'])

seq = (1,2,3,4,5,6)

index.extend(list(itertools.permutations(seq,3)))

writer.writerow(index)

2111

## データ取得開始年月日を指定

In [0]:
#データ取得年月日指定
start = dt.date(2018,1,1)

#取得日数指定
period = 2

In [9]:
i=0

for n in range(period):
    tstr = (start + dt.timedelta(n)).strftime('%Y%m%d')
    print(tstr)
    #試合場でループ
    for k in range(24):
        print(k+1)
        #レースでループ
        for j in range(12):
            #試合結果有無ﾁｪｯｸ
            if k < 9:
                url = "https://sp.macour.jp/s/race/result-info/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"                
            if k >= 9:
                url = "https://sp.macour.jp/s/race/result-info/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"                

            #なぜかfirefoxの体にならないと入れない
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            html = urllib.request.urlopen(req,timeout = 1000)
            soup = BeautifulSoup(html, "html.parser")

            rows = soup.findAll("table", class_="race result2")

            if len(rows) == 0:
                sw1 = 0
                print("大会なし1")
            else:
                sw1 = 1
            #試合番組表有無ﾁｪｯｸ
            if sw1 == 1:
                if k < 9:
                    url = "https://sp.macour.jp/s/race/entry-info/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"                
                if k >= 9:
                    url = "https://sp.macour.jp/s/race/entry-info/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"                
                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("table", class_="race entry")

                if len(rows) == 0:
                    sw2 = 0
                    print("大会なし2")
                else:
                    sw2 = 1
            
            if sw1 == 1 and sw2 == 1:
                if k < 9:
                    url = "https://sp.macour.jp/s/odds/odds-other/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"   
                elif k >= 9:
                    url = "https://sp.macour.jp/s/odds/odds-other/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"

                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("div", class_="col-1")

                if len(rows) == 0:
                    sw3 = 0
                    print("大会なし2")
                else:
                    sw3 = 1
                    
            if sw1 == 1 and sw2 == 1:
                if k < 9:
                    url = "https://sp.macour.jp/s/race/justbefore-info/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"                
                elif k >= 9:
                    url = "https://sp.macour.jp/s/race/justbefore-info/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"               

                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("table", class_="race boats")


                if len(rows) == 0:
                    sw4 = 0
                    print("大会なし2")
                else:
                    sw4 = 1
            
            if sw1 == 1 and sw2 == 1 and sw3 == 1 and sw4 == 1:
                print(j+1)
                odds = []

                odds.append(tstr)
                odds.append(k+1)
                odds.append(j+1)

                ####番組表入力####
                if k < 9:
                    url = "https://sp.macour.jp/s/race/entry-info/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"                
                elif k >= 9:
                    url = "https://sp.macour.jp/s/race/entry-info/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"               

                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("table", class_="race entry")

                entry = []
                #クラス、選手番号
                for cell in rows[0].findAll("td", class_="racer-l-g2"):
                    entry.append(cell.get_text())
                for cell in rows[0].findAll("td", class_="racer-l-y2"):
                    entry.append(cell.get_text())
                for cell in rows[0].findAll("td", class_="racer-l-b2"):
                    entry.append(cell.get_text())
                for cell in rows[0].findAll("td", class_="racer-l-r2"):
                    entry.append(cell.get_text())
                for cell in rows[0].findAll("td", class_="racer-l-k2"):
                    entry.append(cell.get_text())
                for cell in rows[0].findAll("td", class_="racer-l-w2"):
                    entry.append(cell.get_text())

                for i in range(6):
                    odds.extend(entry[i*7+1:i*7+3])
                #全国勝率
                rows = soup.findAll("tr", id="racer-all")

                entry = []
                for cell in rows[0].findAll("td"):
                    entry.append(cell.get_text())               

                odds.extend(entry)

                #全国２率
                rows = soup.findAll("tr", id="racer-all2")

                entry = []
                for cell in rows[0].findAll("td"):
                    a = cell.get_text()
                    a = a.replace('%','')
                    entry.append(a)

                odds.extend(entry)
                #当地勝率
                rows = soup.findAll("tr", id="racer-all-j")

                entry = []
                for cell in rows[0].findAll("td"):
                    entry.append(cell.get_text())

                odds.extend(entry)

                #当地２率
                rows = soup.findAll("tr", id="racer-all2-j")

                entry = []
                for cell in rows[0].findAll("td"):
                    a = cell.get_text()
                    a = a.replace('%','')
                    entry.append(a)

                odds.extend(entry)

                #ST
                rows = soup.findAll("tr", id="racer-st")

                entry = []
                for cell in rows[0].findAll("td"):
                    entry.append(cell.get_text())

                odds.extend(entry)

                #motoer２率
                rows = soup.findAll("tr", id="racer-motor2")

                entry = []
                for cell in rows[0].findAll("td"):
                    a = cell.get_text()
                    a = a.replace('%','')
                    entry.append(a)

                odds.extend(entry)

                #boat２率
                rows = soup.findAll("tr", id="racer-boat2")

                entry = []

                for cell in rows[0].findAll("td"):
                    a = cell.get_text()
                    a = a.replace('%','')
                    entry.append(a)

                odds.extend(entry)

                #####展示情報####
                if k < 9:
                    url = "https://sp.macour.jp/s/race/justbefore-info/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"                
                elif k >= 9:
                    url = "https://sp.macour.jp/s/race/justbefore-info/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"               

                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("table", class_="race boats")

                exibit = []
                for cell in rows[0].findAll("td", class_ = "white"):
                    exibit.append(cell.get_text())

                odds.extend(exibit)

                ####結果データ#####
                if k < 9:
                    url = "https://sp.macour.jp/s/race/result-info/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"                
                elif k >= 9:
                    url = "https://sp.macour.jp/s/race/result-info/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"               

                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("table", class_="race result2")

                result = []

                for cell in rows[0].findAll(['span']):
                    result.append(cell.get_text())

                #着順入力
                for i in range(6):
                    odds.append(int(result[i]))

                ####オッズデータ#####
                #単勝ｵｯｽﾞ
                if k < 9:
                    url = "https://sp.macour.jp/s/odds/odds-other/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"   
                elif k >= 9:
                    url = "https://sp.macour.jp/s/odds/odds-other/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"

                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("div", class_="col-1")

                odds_tan = []
                for cell in rows[0].findAll(['td']):
                    a = cell.get_text()
                    a = a.replace('\t','')
                    a = a.replace('\n','')                   
                    odds_tan.append(a)
                for i in range(6):
                    odds.append(odds_tan[i])

                #3連単ｵｯｽﾞ
                if k < 9:
                    url = "https://sp.macour.jp/s/odds/odds-rentan/date/"+tstr+"/jid/0"+str(k+1)+"/num/"+str(j+1)+"/"   
                elif k >= 9:
                    url = "https://sp.macour.jp/s/odds/odds-rentan/date/"+tstr+"/jid/"+str(k+1)+"/num/"+str(j+1)+"/"   

                #なぜかfirefoxの体にならないと入れない
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                html = urllib.request.urlopen(req,timeout=1000)
                soup = BeautifulSoup(html, "html.parser")

                rows = soup.findAll("div", class_="col-6")

                #試合が無い競艇場はパス
                #選手数分ループ
                for i in range(6):
                    for cell in rows[i].findAll(['td']):
                        odds.append(cell.get_text())

                writer.writerow(odds)

csvFile.close()
print("finish")

20180101
1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
2
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
3
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
4
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
5
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
6
1
2
3
4
5
6
7
8
9
10
11
12
7
1
2
3
4
5
6
7
8
9
10
11
12
8
1
2
3
4
5
6
7
8
9
10
11
12
9
1
2
3
4
5
6
7
8
9
10
11
12
10
1
2
3
4
5
6
7
8
9
10
11
12
11
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
12
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
13
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
14
1
2
3
4
5
6
7
8
9
10
11
12
15
1
2
3
4
5
6
7
8
9
10
11
12
16
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
大会なし1
17
1
2
3
4
5
6
7
8
9
10
11
12
18
1
2
3
4
5
6
7
8
9
10
11
12
19
1
2
3
4
5
6
7
8
9
10
11
12
20
1
2
3
4
5
6
7
8
9
10
1